In [1]:
import sqlite3
import pandas as pd

db_path = 'hot100.db'

table_name = 'hot100'

with sqlite3.connect(db_path) as conn:
    df = pd.read_sql(f"SELECT * FROM {table_name};", conn)

df.head()

,id,artist,song,count,youtube_url,uploader,duration,views,like_count,release_date,thumbnail,tags,description
0,d0f9b947-d9ac-400b-b4dd-25534b30370f,Lady Gaga & Bruno Mars,Die With A Smile,0,https://www.youtube.com/watch?v=kPa7bsKwL-c,Lady Gaga,252,819428697,9289800,,https://i.ytimg.com/vi/kPa7bsKwL-c/maxresdefau...,"[""Lady Gaga"", ""Bruno Mars"", ""Interscope"", ""Pop""]",MAYHEM OUT NOW\nhttp://ladygaga.com \n \nListe...
1,0200f2af-7b0f-4f87-ae4e-fea76fe1305f,Kendrick Lamar,Not Like Us,0,https://www.youtube.com/watch?v=H58vbez_m4E,Kendrick Lamar,354,289684537,6405687,,https://i.ytimg.com/vi/H58vbez_m4E/maxresdefau...,"[""Kendrick Lamar"", ""Kendrick Lamar, under excl...",Kendrick Lamar “Not Like Us”\nDirected by Dave...
2,5d2a6c53-a61c-4024-a173-fd4d39dfe427,Kendrick Lamar Featuring Lefty Gunplay,TV Off,0,https://www.youtube.com/watch?v=4ILLpWbgESM,Certified Filmmaker,221,1092830,14234,20241221,https://i.ytimg.com/vi/4ILLpWbgESM/sddefault.j...,"[""Kendrick Lamar"", ""tv off"", ""GNX"", ""Music"", ""...",#kendricklamar #tvoff #gnx #music #musicvideo ...
3,34ce80c8-b2de-41e3-a721-bffb69e15c07,Shaboozey,A Bar Song (Tipsy),0,https://www.youtube.com/watch?v=t7bQwwqW-Hc,Shaboozey,173,204128472,1625863,,https://i.ytimg.com/vi/t7bQwwqW-Hc/maxresdefau...,"[""Shaboozey"", ""Bar"", ""Song"", ""(Tipsy)"", ""[Offi...","Listen to the single “A Bar Song (Tipsy)"". Out..."
4,e3201b4d-5a39-45b0-8bf3-0e5eeff9894a,ROSE & Bruno Mars,APT.,0,https://www.youtube.com/watch?v=ekr2nIex040,ROSÉ,173,1298483960,14699729,20241018,https://i.ytimg.com/vi_webp/ekr2nIex040/maxres...,"[""YG Entertainment"", ""YG"", ""\uc640\uc774\uc9c0...",ROSÉ & Bruno Mars - APT.\nDownload/stream: ht...
